# Testing module

In [1]:
import csv
import os
import pandas as pd
import pickle 
import numpy as np
import mediapipe as mp
import cv2
import datetime

In [2]:
prj_path = r"C:\Users\Chale\Desktop\Project ITM65\Model Framework\Code\Web-applications-for-elderly-physical-exercises\ElderFitmodel"

In [3]:
model_name = 'handup_model'
model_dst=prj_path+"\Model/"
save_dst=model_dst+model_name+'.pkl'
model = pickle.load(open(save_dst, 'rb'))

In [4]:
# Grabbing the Holistic Model from Mediapipe and
mp_holistic = mp.solutions.holistic

# Initializing the Model
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [5]:
#pth_pic = r"C:\Users\Chale\Desktop\Project ITM65\Model Framework\Code\Web-applications-for-elderly-physical-exercises\ElderFitmodel\Dataset\handup_model\handup_pic\\"

In [6]:
# #load picture
# img_normal = cv2.imread(pth_pic+"normal.jpg")
# img_lefthand = cv2.imread(pth_pic+"lefthand.jpg")
# img_righthand = cv2.imread(pth_pic+"righthand.jpg")
# img_bothhand = cv2.imread(pth_pic+"both.jpg")

In [7]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
status = "normal"
timegetter = 0
act = True

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()

    # resizing the frame
    frame = cv2.resize(frame, (880,720))
        
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    
    results = holistic_model.process(image)
    
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
    )

    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )

    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,   
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        # Make Detections
        X = pd.DataFrame([pose_row])
        predict_class = model.predict(X)[0]
        predict_prob = model.predict_proba(X)[0]
        class_name = predict_class.split(' ')[0]

        coords = tuple(np.multiply(
        np.array(
            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
             results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
        , [640,480]).astype(int))
            
        cv2.rectangle(image, 
                      (coords[0], coords[1]+5), 
                      (coords[0]+len(predict_class)*20, coords[1]-30), 
                      (245, 117, 16), -1)
        cv2.putText(image, predict_class, coords, 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        
#         # Get status box
#         cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

#         # Display Class
#         cv2.putText(image, 'CLASS'
#                     , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, predict_class.split(' ')[0]
#                     , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#         # Display Probability
#         cv2.putText(image, 'PROB'
#                     , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, str(round(predict_prob[np.argmax(predict_prob)],2))
#                     , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    except:
        cv2.putText(image, str('Error')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    timegetter+=1
    # Display the resulting image
    cv2.imshow("Test Model", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') :
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()